# Data analysis
2019.04.06 - ?

I get to data analysis in this notebook. This is _part 1_, as I am planning on attacking analysis from another aspect in the future. This notebook looks at the usage of articles in front of specific count and mass nouns, obtained from the prepatory analysis and exploration. However, due to limitations on finding nouns used in each level of interest for four language groups in two different corpora (Arabic L1 from BALC, Arabic L1 from PELIC, Spanish L1 from PELIC, Korean L1 from PELIC), only 1 sample from each level in each language was able to be obtained. Due to this, and due to the fact that I may not have very many interesting or informative findings, I am planning on expanding my analysis in a second part.

## Summary of code
- 

### Set-up

In [1]:
import pandas as pd
import numpy as np
import re 

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

%pprint            # to turn off pretty printing

Pretty printing has been turned OFF


In [2]:
# I tried to use .read_csv, but it didn't work, so I'll continue to use .from_csv
balc = pd.DataFrame.from_csv('../data/balc_targets.csv', index_col=[0])
pelic = pd.DataFrame.from_csv('../private/pelic_targets.csv', index_col=[0])
targets = pd.DataFrame.from_csv('../exploratory-analysis/chosen_targets_new.csv', index_col=[0])

/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  This is separate from the ipykernel package so we can avoid doing imports until
/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: FutureWarning: from_csv is deprecated. Please use read_csv(...) instead. Note that some of the default arguments are different, so please refer to the documentation for from_csv when changing your function calls
  after removing the cwd from sys.path.


In [3]:
balc.head()

,index,Filename,Level,Original_Text,Normalized_Essay,Revised_Essay,tokens,token_count,TTR,Guiraud,pos,lemmas,targ
0,1347,200611403,3,\t\t\t\tCEPA 3 200611403\n\n\n\nLast summer ho...,Last summer holiday iam going with my family t...,Last summer holiday iam going with my family t...,"['Last', 'summer', 'holiday', 'iam', 'going', ...",82,0.536585,4.858987,"[(Last, 'JJ'), (summer, 'NN'), (holiday, 'NN')...","['last', 'summer', 'holiday', 'iam', 'go', 'wi...",book
1,1171,200608092,4,\t\t\t\tCEPA 4 200608092\n\n\n\nI have just ha...,I have just had the perfect holidy. In my holi...,I have just had the perfect holidy. In my holi...,"['I', 'have', 'just', 'had', 'the', 'perfect',...",188,0.446809,6.126330,"[(I, 'PRP'), (have, 'VBP'), (just, 'RB'), (had...","['-PRON-', 'have', 'just', 'have', 'the', 'per...",book
2,1441,200603618,5,\t\t\t\tCEPA 5 200603618\n\n\n\nIn the summer ...,In the summer holiday my family dicides to spe...,In the summer holiday my family dicides to spe...,"['In', 'the', 'summer', 'holiday', 'my', 'fami...",215,0.460465,6.751744,"[(In, 'IN'), (the, 'DT'), (summer, 'NN'), (hol...","['in', 'the', 'summer', 'holiday', '-PRON-', '...",book
3,1530,200607320,6,\t\t\t\tCEPA 6 200607320\n\n\n\nThere have bee...,"There have been many films reteased lately, So...","There have been many films reteased lately, So...","['There', 'have', 'been', 'many', 'films', 're...",234,0.598291,9.152086,"[(There, 'EX'), (have, 'VBP'), (been, 'VBN'), ...","['there', 'have', 'be', 'many', 'film', 'retea...",book
4,881,200607798,3,\t\t\t\tCEPA 3 200607798\n\nWhere I going to t...,"Where I going to the India with my family , my...","Where I going to the India with my family , my...","['Where', 'I', 'going', 'to', 'the', 'India', ...",151,0.602649,7.405475,"[(Where, 'WRB'), (I, 'PRP'), (going, 'VBG'), (...","['where', '-PRON-', 'go', 'to', 'the', 'india'...",money


In [4]:
# drop the index column, we don't need it
balc = balc.drop('index', axis=1)
balc.head()

,Filename,Level,Original_Text,Normalized_Essay,Revised_Essay,tokens,token_count,TTR,Guiraud,pos,lemmas,targ
0,200611403,3,\t\t\t\tCEPA 3 200611403\n\n\n\nLast summer ho...,Last summer holiday iam going with my family t...,Last summer holiday iam going with my family t...,"['Last', 'summer', 'holiday', 'iam', 'going', ...",82,0.536585,4.858987,"[(Last, 'JJ'), (summer, 'NN'), (holiday, 'NN')...","['last', 'summer', 'holiday', 'iam', 'go', 'wi...",book
1,200608092,4,\t\t\t\tCEPA 4 200608092\n\n\n\nI have just ha...,I have just had the perfect holidy. In my holi...,I have just had the perfect holidy. In my holi...,"['I', 'have', 'just', 'had', 'the', 'perfect',...",188,0.446809,6.126330,"[(I, 'PRP'), (have, 'VBP'), (just, 'RB'), (had...","['-PRON-', 'have', 'just', 'have', 'the', 'per...",book
2,200603618,5,\t\t\t\tCEPA 5 200603618\n\n\n\nIn the summer ...,In the summer holiday my family dicides to spe...,In the summer holiday my family dicides to spe...,"['In', 'the', 'summer', 'holiday', 'my', 'fami...",215,0.460465,6.751744,"[(In, 'IN'), (the, 'DT'), (summer, 'NN'), (hol...","['in', 'the', 'summer', 'holiday', '-PRON-', '...",book
3,200607320,6,\t\t\t\tCEPA 6 200607320\n\n\n\nThere have bee...,"There have been many films reteased lately, So...","There have been many films reteased lately, So...","['There', 'have', 'been', 'many', 'films', 're...",234,0.598291,9.152086,"[(There, 'EX'), (have, 'VBP'), (been, 'VBN'), ...","['there', 'have', 'be', 'many', 'film', 'retea...",book
4,200607798,3,\t\t\t\tCEPA 3 200607798\n\nWhere I going to t...,"Where I going to the India with my family , my...","Where I going to the India with my family , my...","['Where', 'I', 'going', 'to', 'the', 'India', ...",151,0.602649,7.405475,"[(Where, 'WRB'), (I, 'PRP'), (going, 'VBG'), (...","['where', '-PRON-', 'go', 'to', 'the', 'india'...",money


In [5]:
pelic.head()

,answer_id,question_id,anon_id,user_file_id,text,class_code,level_id,native_language,version,toks_re,toks_re_len,toks_nltk,targ
0,1897,188,am1,13293,Adjective\n a lot of traditional desserts\n su...,w,3,Arabic,1,"['adjective', 'a', 'lot', 'of', 'traditional',...",200,"['Adjective', 'a', 'lot', 'of', 'traditional',...",book
1,31692,4194,du3,69515,"I like book called The Last Lecture,because it...",w,4,Arabic,1,"['i', 'like', 'book', 'called', 'the', 'last',...",13,"['I', 'like', 'book', 'called', 'The', 'Last',...",book
2,36798,4778,dp7,78306,Studying English is very crucial because it ...,w,5,Arabic,1,"['studying', 'english', 'is', 'very', 'crucial...",132,"['Studying', 'English', 'is', 'very', 'crucial...",book
3,18572,2544,dg7,41011,? LIST\n\nThe entrance is in front of t...,w,3,Korean,1,"['list', 'the', 'entrance', 'is', 'in', 'front...",124,"['?', 'LIST', 'The', 'entrance', 'is', 'in', '...",book
4,1483,140,eu6,11952,1.The !Kung's most important food is the mongo...,w,4,Korean,1,"['the', 'kung', 's', 'most', 'important', 'foo...",164,"['1', '.', 'The', '!', 'Kung', ""'s"", 'most', '...",book


### Testing regex
I'll be using regular expressions to look for the targets, so below is some testing I did on a UDF and some test phrases to see what I can do with this.

In [6]:
myre = re.compile(r'((\w+ ){,2}\w+ book.*?\b)', re.I)
test = ["I put the book on the counter.", "I put a book on the counter.", "I put the books on the counter",
       "I put some books on the counter.", "I put my books on the counter.", "I put book on the counter.",
       "I put books on the counter.", "I put book and books and the other books on the counter."]

def getArticles(regex, text):
    myre = re.compile(regex)
    search = myre.findall(text)
    articles = []
    for item in search:
        if 'the' in item:
            articles.append((item, 'the'))
        elif 'a' in item or 'an' in item:
            articles.append((item, 'a/an'))
        else:
            articles.append((item, '0'))
    return articles

In [7]:
for s in test:
    print(getArticles(myre, s))

[(('I put the book', 'put '), '0')]
[(('I put a book', 'put '), '0')]
[(('I put the books', 'put '), '0')]
[(('I put some books', 'put '), '0')]
[(('I put my books', 'put '), '0')]
[(('I put book', 'I '), '0')]
[(('I put books', 'I '), '0')]
[(('I put book', 'I '), '0'), (('and books', ''), '0'), (('and the other books', 'the '), '0')]


Well, this is an obvious issue. All of the returns are zero articles, which is simply not true! Let's investigate a little further...

In [8]:
s1 = test[0]
a = myre.findall(s1)
a

[('I put the book', 'put ')]

In [9]:
a[0]
'the' in a[0]

a[0][0]
'the' in a[0][0]

('I put the book', 'put ')

False

'I put the book'

True

So the issue was that I was initially targeting the entire tuple, not the string object of interest, and it was returning 'false' for everything I searched for, thus kicking everything down to the 'else' condition. Let's rewrite getArticles!

In [10]:
def getArticles(regex, text):
    """Uses a regex targeting a noun. Finds all instances of the noun and returns whether the return contains a definite article, an indefinite article, a zero article, or other types of determiners.
    Input: regex (a regex compile), text (can be a single string object or a list of string objects).
    Returns item and type of article in a list format."""
    myre = re.compile(regex)
    look = myre.findall(text)
    other = ['my', 'his', 'her', 'their', 'their', 'our', 'ours', 'its', 
            'this', 'that', 'these', 'those', 'some', 'much', 'many', 'most', 'some', 'any', 'enough',
            'all', "both", 'half', 'either', 'neither', 'each', 'every']
    articles = []
    indef = re.compile(r'\ban?\b')
    for item in look:
        if any(search in item[0] for search in other):
            if 'the' in item[0]:
                articles.append((item, 'the'))
            elif indef.search(item[0]):
                matchobj = indef.search(item[0])
                if matchobj:
                    articles.append((item, 'a/an'))
                else:
                    articles.append((item, 'other'))
            else:
                articles.append((item, 'other'))
        elif 'the' in item[0]:
            articles.append((item, 'the'))
        elif indef.search(item[0]):
            matchobj = indef.search(item[0])
            if matchobj:
                articles.append((item, 'a/an'))
        else:
            articles.append((item, '0'))
    return articles

In [11]:
for s in test:
    print(getArticles(myre, s))

[(('I put the book', 'put '), 'the')]
[(('I put a book', 'put '), 'a/an')]
[(('I put the books', 'put '), 'the')]
[(('I put some books', 'put '), 'other')]
[(('I put my books', 'put '), 'other')]
[(('I put book', 'I '), '0')]
[(('I put books', 'I '), '0')]
[(('I put book', 'I '), '0'), (('and books', ''), '0'), (('and the other books', 'the '), 'the')]


In [12]:
getArticles(myre, s1)

[(('I put the book', 'put '), 'the')]

In [13]:
search = myre.findall(pelic.text[0])
search

[('and international dessert books', 'international '), ('many of dessert books', 'of '), ('and international dessert books', 'international ')]

In [14]:
getArticles(myre, pelic.text[0])

[(('and international dessert books', 'international '), '0'), (('many of dessert books', 'of '), 'other'), (('and international dessert books', 'international '), '0')]

It may not be perfect, so we'll need to double-check along the way, but this should be a good start. Let's start looking at some nouns!

### Count nouns
In this section, I'll take a look at the targets that are count nouns:
- book 
- place
- park
- week
- story

In [15]:
targets[targets.type == 'count']

,lemma,counts_cepa,counts_pelic,type
37,place,770,1883.0,count
73,park,431,247.0,count
101,week,308,469.0,count
154,story,182,254.0,count
194,book,121,685.0,count


#### 'Book'
myre is already set to look for 'book', so let's just jump into it!

In [16]:
book_pelic = pelic[pelic.targ == 'book']
book_balc = balc[balc.targ == 'book']

In [17]:
book_pelic

,answer_id,question_id,anon_id,user_file_id,text,class_code,level_id,native_language,version,toks_re,toks_re_len,toks_nltk,targ
0,1897,188,am1,13293,Adjective\n a lot of traditional desserts\n su...,w,3,Arabic,1,"['adjective', 'a', 'lot', 'of', 'traditional',...",200,"['Adjective', 'a', 'lot', 'of', 'traditional',...",book
1,31692,4194,du3,69515,"I like book called The Last Lecture,because it...",w,4,Arabic,1,"['i', 'like', 'book', 'called', 'the', 'last',...",13,"['I', 'like', 'book', 'called', 'The', 'Last',...",book
2,36798,4778,dp7,78306,Studying English is very crucial because it ...,w,5,Arabic,1,"['studying', 'english', 'is', 'very', 'crucial...",132,"['Studying', 'English', 'is', 'very', 'crucial...",book
3,18572,2544,dg7,41011,? LIST\n\nThe entrance is in front of t...,w,3,Korean,1,"['list', 'the', 'entrance', 'is', 'in', 'front...",124,"['?', 'LIST', 'The', 'entrance', 'is', 'in', '...",book
4,1483,140,eu6,11952,1.The !Kung's most important food is the mongo...,w,4,Korean,1,"['the', 'kung', 's', 'most', 'important', 'foo...",164,"['1', '.', 'The', '!', 'Kung', ""'s"", 'most', '...",book
5,2738,300,ad1,16211,The statue is situated in the middle of a sea ...,w,5,Korean,1,"['the', 'statue', 'is', 'situated', 'in', 'the...",60,"['The', 'statue', 'is', 'situated', 'in', 'the...",book
6,29216,3881,bt8,66247,...,w,3,Spanish,1,"['types', 'of', 'magazines', 'there', 'are', '...",166,"['Types', 'of', 'magazines', 'There', 'are', '...",book
7,18537,2541,en1,40609,You can get other personal reasons in order to...,w,4,Spanish,1,"['you', 'can', 'get', 'other', 'personal', 're...",90,"['You', 'can', 'get', 'other', 'personal', 're...",book
8,33911,4461,bi4,73652,Learning English has been very hard to underst...,w,5,Spanish,1,"['learning', 'english', 'has', 'been', 'very',...",365,"['Learning', 'English', 'has', 'been', 'very',...",book


In [18]:
book_balc

,Filename,Level,Original_Text,Normalized_Essay,Revised_Essay,tokens,token_count,TTR,Guiraud,pos,lemmas,targ
0,200611403,3,\t\t\t\tCEPA 3 200611403\n\n\n\nLast summer ho...,Last summer holiday iam going with my family t...,Last summer holiday iam going with my family t...,"['Last', 'summer', 'holiday', 'iam', 'going', ...",82,0.536585,4.858987,"[(Last, 'JJ'), (summer, 'NN'), (holiday, 'NN')...","['last', 'summer', 'holiday', 'iam', 'go', 'wi...",book
1,200608092,4,\t\t\t\tCEPA 4 200608092\n\n\n\nI have just ha...,I have just had the perfect holidy. In my holi...,I have just had the perfect holidy. In my holi...,"['I', 'have', 'just', 'had', 'the', 'perfect',...",188,0.446809,6.126330,"[(I, 'PRP'), (have, 'VBP'), (just, 'RB'), (had...","['-PRON-', 'have', 'just', 'have', 'the', 'per...",book
2,200603618,5,\t\t\t\tCEPA 5 200603618\n\n\n\nIn the summer ...,In the summer holiday my family dicides to spe...,In the summer holiday my family dicides to spe...,"['In', 'the', 'summer', 'holiday', 'my', 'fami...",215,0.460465,6.751744,"[(In, 'IN'), (the, 'DT'), (summer, 'NN'), (hol...","['in', 'the', 'summer', 'holiday', '-PRON-', '...",book
3,200607320,6,\t\t\t\tCEPA 6 200607320\n\n\n\nThere have bee...,"There have been many films reteased lately, So...","There have been many films reteased lately, So...","['There', 'have', 'been', 'many', 'films', 're...",234,0.598291,9.152086,"[(There, 'EX'), (have, 'VBP'), (been, 'VBN'), ...","['there', 'have', 'be', 'many', 'film', 'retea...",book


In [19]:
getArticles(myre, book_pelic.text[0])
book_pelic.text[0]

[(('and international dessert books', 'international '), '0'), (('many of dessert books', 'of '), 'other'), (('and international dessert books', 'international '), '0')]

"Adjective\n a lot of traditional desserts\n superior success\n My mother was a wonderful cook\n national and international dessert books\n practiced the desirable recipes\n my favorite food channels\n\n   Phrases of prepositions\n from my mother\n many of dessert books\n on T.V for half an hour a day\n\n\n      Topic:  personal success\n            Making a dessert\n\n        I think I have a personal success in making a lot of traditional desserts from several countries.  There were some reasons for helping my superior success. One of the reasons, I gained this talent from my mother.  She was a wonderful cook especially when she made cakes. In fact, my mother taught me how to prepare German desserts.  For example, I learned the crepes, many kinds of the nut cakes, fruit cakes, and the cake's topping, too. Another reason for my success, I read many of national and international dessert books for several times ago.  I practiced the desirable recipes as soon as I read about them.  Final

In [22]:
index = 0
for x in book_pelic.text:
    print("book_pelic:", index)
    print(getArticles(myre, x))
    print(x)
    print()
    index += 1

book_pelic: 0
[(('and international dessert books', 'international '), '0'), (('many of dessert books', 'of '), 'other'), (('and international dessert books', 'international '), '0')]
Adjective
 a lot of traditional desserts
 superior success
 My mother was a wonderful cook
 national and international dessert books
 practiced the desirable recipes
 my favorite food channels

   Phrases of prepositions
 from my mother
 many of dessert books
 on T.V for half an hour a day


      Topic:  personal success
            Making a dessert

        I think I have a personal success in making a lot of traditional desserts from several countries.  There were some reasons for helping my superior success. One of the reasons, I gained this talent from my mother.  She was a wonderful cook especially when she made cakes. In fact, my mother taught me how to prepare German desserts.  For example, I learned the crepes, many kinds of the nut cakes, fruit cakes, and the cake's topping, too. Another reason 

In [23]:
index = 0
for x in book_balc.Revised_Essay:
    print("book_balc:", index)
    print(getArticles(myre, x))
    print(x)
    print()
    index += 1

book_balc: 0
[(('wooke in the booke', 'in '), 'the')]
Last summer holiday iam going with my family to perfect weekend. we going to the India. We going to wooke in the booke and go to shopping. And we will go to see Tag Mhale. It was so wonderful. cause it have a nice people and a nice shope. After I finsed in India iam going to Hangare with my family. we spend in Loxx. And iam plaing football and going to swam.

book_balc: 1
[(('friend reading some Bookes', 'reading '), 'other')]
I have just had the perfect holidy. In my holiday I went to dubai, Abu dabi, Sharjah Al-Ain and Fujirah I went there places with my family and my friends. I went the cinama and watched nice movies with my friends, I went malls I buys clothes and play some time with sisters and brothers for example went the sea with my family I just had a wonderfor time I swam in the sea and play with my father and sisters football. I'm very happy becouse I have nice holiday. In a some time I went net coffe set in a nel and dra